# Laboratory №6

### Task 1

$$ I = \int\int_0 f(x, y) dxdy $$

In [22]:
import csv
import numpy as np

file = "input.csv"

def readFromFile(file):

    Table = []

    with open(file) as f:
        reader = csv.DictReader(f)

        for row in reader:
            Table.append(np.array(list(map(float, [*row.values()]))))

    return Table

Table = np.array(readFromFile(file))



[[0.     0.     5.     4.0937 3.3516 2.7441 2.2466 1.8394 1.506  1.233
  1.0095 0.8265 0.6767 0.554  0.4536]
 [0.1    0.1    3.7041 3.0327 2.4829 2.0328 1.6644 1.3627 1.1157 0.9134
  0.7478 0.6123 0.5013 0.4104 0.336 ]
 [0.2    0.2    2.7441 2.2466 1.8394 1.506  1.233  1.0095 0.8265 0.6767
  0.554  0.4536 0.3714 0.3041 0.2489]
 [0.3    0.3    2.0328 1.6644 1.3627 1.1157 0.9134 0.7478 0.6123 0.5013
  0.4104 0.336  0.2751 0.2252 0.1844]
 [0.4    0.4    1.506  1.233  1.0095 0.8265 0.6767 0.554  0.4536 0.3714
  0.3041 0.2489 0.2038 0.1669 0.1366]
 [0.5    0.5    1.1157 0.9134 0.7478 0.6123 0.5013 0.4104 0.336  0.2751
  0.2252 0.1844 0.151  0.1236 0.1012]
 [0.6    0.6    0.8265 0.6767 0.554  0.4536 0.3714 0.3041 0.2489 0.2038
  0.1669 0.1366 0.1119 0.0916 0.075 ]
 [0.7    0.7    0.6123 0.5013 0.4104 0.336  0.2751 0.2252 0.1844 0.151
  0.1236 0.1012 0.0829 0.0678 0.0555]
 [0.8    0.8    0.4536 0.3714 0.3041 0.2489 0.2038 0.1669 0.1366 0.1119
  0.0916 0.075  0.0614 0.0503 0.0411]
 [0.9    0.9

### Task 2

In [46]:
import numpy as np
import prettytable as pt

N = 6

def formatOut(num):
    return f"{num:.5f}"

def leftDiffDer(yValue, step, index):
    if index > 0 and index < len(yValue):
        return formatOut((yValue[index] - yValue[index - 1]) / step)
    else:
        return '*'

def centerDiffDer(yValue, step, index):
    if index > 0 and index < len(yValue) - 1:
        return formatOut((yValue[index + 1] - yValue[index - 1]) / (2 * step))
    else:
        return '*'

def secondRunge(yValue, step, index):
    if index < 2:
        return '*'
        
    f1 = (yValue[index] - yValue[index - 1]) / (2 * step)
    f2 = (yValue[index] - yValue[index - 2]) / (2 * step)

    return formatOut(2 * f1 - f2)

def aligVars(yValue, xValue, index):
    if index > len(yValue) - 2:
        return '*'

    d = (1 / yValue[index + 1] - 1 / yValue[index]) / \
        (1 / xValue[index + 1] - 1 / xValue[index])

    return formatOut(d * yValue[index] ** 2 / xValue[index] ** 2)

def secondDiffDer(yValue, step, index):
    if index > 0 and index < len(yValue) - 1:
        return formatOut((yValue[index - 1] - 2 * yValue[index] + yValue[index + 1]) / step ** 2)
    else:
        return ' *'

xArr = [1, 2, 3, 4, 5, 6]
yArr = [0.571, 0.889, 1.091, 1.231, 1.333, 1.412]
    
step = (xArr[-1] - xArr[0]) / len(xArr)

methods = [leftDiffDer, centerDiffDer, 
           secondRunge, aligVars, 
           secondDiffDer]

In [47]:
table = pt.PrettyTable()
filedNames = ["X", "Y", "1", "2", "3", "4", "5"]

table.add_column(filedNames[0], xArr)
table.add_column(filedNames[1], yArr)

for i in range(len(methods)):
    if i == 3:
        table.add_column(filedNames[i + 1], [methods[i](yArr, xArr, j) for j in range(N)])
    else:    
        table.add_column(filedNames[i + 2], [methods[i](yArr, step, j) for j in range(N)])

print(table)

+---+-------+---------+---------+----------+---------+----------+
| X |   Y   |    1    |    2    |    3     |    3    |    5     |
+---+-------+---------+---------+----------+---------+----------+
| 1 | 0.571 |    *    |    *    |    *     | 0.40850 |     *    |
| 2 | 0.889 | 0.38160 | 0.31200 |    *     | 0.24690 | -0.16704 |
| 3 | 1.091 | 0.24240 | 0.20520 | -0.06960 | 0.16544 | -0.08928 |
| 4 | 1.231 | 0.16800 | 0.14520 | -0.03720 | 0.11774 | -0.05472 |
| 5 | 1.333 | 0.12240 | 0.10860 | -0.02280 | 0.08950 | -0.03312 |
| 6 | 1.412 | 0.09480 |    *    | -0.01380 |    *    |     *    |
+---+-------+---------+---------+----------+---------+----------+
